# Chatbot for Legal Exam Queries

This notebook demonstrates how to build a simple chatbot to answer frequently asked questions about the CLAT (Common Law Admission Test) exam.


In [1]:
#Import Libraries
import pandas as pd
import re
import torch
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer, util
from parrot import Parrot
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Utility Functions
# Clean text by removing special characters and lowercasing
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", "", text)
    return text.strip()

# Extract visible text from a webpage
def extract_full_text(url):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, "html.parser")
        for script in soup(["script", "style"]): 
            script.decompose()
        text = soup.get_text(separator="\n")
        lines = [line.strip() for line in text.splitlines() if line.strip()]
        return "\n".join(lines)
    except Exception as e:
        return f"Failed to extract from {url}: {str(e)}"
 
 # Split text into smaller word-based chunks
def chunk_text(text, chunk_size=30):
    words = text.split()
    chunks = [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

In [3]:
# Load the Dataset
# Load the pre-generated dataset (with original + paraphrased questions)
df_expanded = pd.read_csv("clat_qa_dataset_with_paraphrases.csv")

# Clean (if not already cleaned in the file)
df_expanded["clean_question"] = df_expanded["question"].apply(clean_text)

# Preview
df_expanded.head()

,question,answer,clean_question
0,What is the syllabus for CLAT 2025?,The CLAT 2025 syllabus includes English Langua...,what is the syllabus for clat 2025
1,How many questions are there in the English se...,The English Language section typically has aro...,how many questions are there in the english se...
2,What was the cutoff for NLSIU Bangalore last y...,The CLAT 2024 cutoff rank for NLSIU Bangalore ...,what was the cutoff for nlsiu bangalore last year
3,What is the total duration of the CLAT exam?,The CLAT exam duration is 2 hours (120 minutes).,what is the total duration of the clat exam
4,What is the marking scheme in CLAT?,Each correct answer gives you 1 mark and 0.25 ...,what is the marking scheme in clat


In [4]:
#Web Scraping for Additional CLAT Info
urls = [
    "http://www.clpd.resonance.ac.in/studentzone/faq-CLAT.aspx",
    "https://consortiumofnlus.ac.in/clat-2025/FAQs.html",
    "https://knowledgenation.co.in/FAQ'S_General",
    "https://law.careers360.com/articles/clat-faqs",
    "https://vidhigya.com/blog/frequently-asked-questions-about-clat/"
]

extracted_data = []
for url in urls:
    content = extract_full_text(url)
    extracted_data.append({"url": url, "text": content})

df_web = pd.DataFrame(extracted_data)
df_web["clean_text"] = df_web["text"].apply(clean_text)

# Save web data
df_web.to_csv("faq_web_data.csv", index=False)

In [5]:
#Chunk and Embed Web Text
all_chunks = []
for _, row in df_web.iterrows():
    chunks = chunk_text(row["clean_text"])
    all_chunks.extend(chunks)

df_chunks = pd.DataFrame({"chunk": all_chunks})

In [6]:
# Load Embedding Model & Generate Embeddings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentenceTransformer("all-MiniLM-L6-v2", device=device)

# Generate Embeddings
question_embeddings = model.encode(df_expanded['clean_question'].tolist(), convert_to_tensor=True)
web_embeddings = model.encode(df_chunks['chunk'].tolist(), convert_to_tensor=True)

In [7]:
#Define Answer Retrieval Logic
def get_answer_with_web_fallback(user_query, df_faq, faq_embeddings, df_chunks, web_embeddings, model, threshold=0.65):
    cleaned_query = clean_text(user_query)
    query_embedding = model.encode(cleaned_query, convert_to_tensor=True)

    # Match from FAQ dataset
    faq_similarities = util.pytorch_cos_sim(query_embedding, faq_embeddings)
    top_faq_idx = torch.argmax(faq_similarities)
    top_faq_score = faq_similarities[0][top_faq_idx].item()

    if top_faq_score >= threshold:
        return df_faq.iloc[top_faq_idx.item()]["answer"]

    # Fallback: Match from web scraped chunks
    web_similarities = util.pytorch_cos_sim(query_embedding, web_embeddings)
    top_web_idx = torch.argmax(web_similarities)
    top_web_score = web_similarities[0][top_web_idx].item()

    if top_web_score >= threshold:
        return df_chunks.iloc[top_web_idx.item()]["chunk"]

    return "I'm sorry, I don't have an answer to that question at the moment. You can try rephrasing it or visit the official CLAT website for more information."

In [8]:
# Example 1
query = "What’s the maximum age to appear for CLAT?"
answer = get_answer_with_web_fallback(query, df_expanded, question_embeddings, df_chunks, web_embeddings, model)
print("User:", query)
print("Bot:", answer)

# Example 2
query = "What is CLAT?"
answer = get_answer_with_web_fallback(query, df_expanded, question_embeddings, df_chunks, web_embeddings, model)
print("\nUser:", query)
print("Bot:", answer)

# Example 3
query = "I have done engineering. Can I appear in CLAT?"
answer = get_answer_with_web_fallback(query, df_expanded, question_embeddings, df_chunks, web_embeddings, model)
print("\nUser:", query)
print("Bot:", answer)

User: What’s the maximum age to appear for CLAT?
Bot: No, there is no upper age limit for appearing in the CLAT exam.

User: What is CLAT?
Bot: CLAT stands for Common Law Admission Test. It is an entrance exam conducted all over India for admission into various law courses in participating law schools. CLAT is conducted for undergraduate students (5 year dual degree programs which have LLB and degrees awarded from BA to BSc, BBA, and B.Com) as well as for Graduate students opting for LLM programs.

User: I have done engineering. Can I appear in CLAT?
Bot: I'm sorry, I don't have an answer to that question at the moment. You can try rephrasing it or visit the official CLAT website for more information.
